In [32]:
import pandas as pd
from warnings import filterwarnings
filterwarnings("ignore")

In [3]:
tsd=pd.read_csv("/content/testing_set.csv")
tsd.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
trd=pd.read_csv("/content/training_set.csv")
trd.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Missing Data Treatment

In [5]:
Q=pd.DataFrame(trd.isna().sum(),columns=["ct"])
Q[Q.ct > 0].index

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [6]:
Q = pd.DataFrame(tsd.isna().sum(),columns=["ct"])
Q[Q.ct > 0].index

Index(['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath',
       'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType'],
      dtype='object')

In [7]:
def replacer(df):
    Q = pd.DataFrame(df.isna().sum(),columns=['count_mv'])
    R = Q[Q.count_mv > 0]
    cat = []
    con = []
    for i in R.index:
        if(df[i].dtypes == 'object'):
            replacer = df[i].mode()[0]
            df[i] = df[i].fillna(replacer)
        else:
            replacer = round(df[i].mean(),2)
            df[i] = df[i].fillna(replacer)

In [8]:
replacer(trd)
replacer(tsd)

Droping Unnecessary Columns

In [9]:
trd1 = trd.drop(labels=["Id"],axis=1)
tsd1 = tsd.drop(labels=["Id"],axis=1)

Find Cat and Con Columns

In [13]:
cat = []
con = []
for i in trd1.columns:
    if(trd1[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

Bivariate Analysis

In [10]:
Q = trd1.corr()[["SalePrice"]]
con_cols_to_consider = list(Q[Q.SalePrice > 0.3].sort_values(by="SalePrice",ascending =False).index)
con_cols_to_consider.remove("SalePrice")

In [11]:
def ANOVA(df,cat,con):
    from pandas import DataFrame
    from statsmodels.api import OLS
    from statsmodels.formula.api import ols
    rel = con + " ~ " + cat
    model = ols(rel,df).fit()
    from statsmodels.stats.anova import anova_lm
    anova_results = anova_lm(model)
    Q = DataFrame(anova_results)
    a = Q['PR(>F)'][cat]
    return round(a,3)

In [14]:
W = []
for i in cat:
    W.append(ANOVA(trd1,i,"SalePrice"))

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [15]:
Q = pd.DataFrame([cat,W]).T
Q.columns = ["column","pval"]

In [16]:
cat_cols_to_consider = list(Q[Q.pval < 0.1].column)

Preprocessing of training Set

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
tr1 = trd1[cat_cols_to_consider].apply(le.fit_transform)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
tr2 = pd.DataFrame(ss.fit_transform(trd1[con_cols_to_consider]),columns=con_cols_to_consider)

In [18]:
trd3 = tr2.join(tr1)

In [19]:
trd3.head(2)

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,...,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,SaleType,SaleCondition
0,0.651479,0.370333,0.311725,0.351000,-0.459303,-0.793434,0.789741,0.912210,1.050994,0.878668,...,2,1,1,4,4,2,2,2,8,4
1,-0.071836,-0.482512,0.311725,-0.060731,0.466465,0.257140,0.789741,-0.318683,0.156734,-0.429577,...,4,1,1,4,4,2,2,2,8,4


In [20]:
trd3.shape

(1460, 56)

Preprocessing of Prediction Set

In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
ts1 = tsd1[cat_cols_to_consider].apply(le.fit_transform)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ts2 = pd.DataFrame(ss.fit_transform(tsd1[con_cols_to_consider]),columns = con_cols_to_consider)

In [22]:
pred_set = ts2.join(ts1)
pred_set.shape

(1459, 56)

Define X and Y,divide the data in training and testing set for model creation purpose

In [23]:
X = trd3
Y = trd[["SalePrice"]]
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.2,random_state=31)

In [24]:
from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst).fit()
ols.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.854
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     116.4
Date:                Fri, 04 Mar 2022   Prob (F-statistic):               0.00
Time:                        22:55:22   Log-Likelihood:                -13680.
No. Observations:                1168   AIC:                         2.747e+04
Df Residuals:                    1111   BIC:                         2.776e+04
Df Model:                          56                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          3.227e+05   2.84e+04     11.349      0.000    2.67e+05    3.78e+05
OverallQual    1.635e+04   1682.759      9.714      0.000     1.3e+04    1.96e+04
GrLivArea      7354.5543   1.07e+04      0.689      0.491   -1.36e+04    2.83e+04
GarageCars     9974.9725   2160.510      4.617      0.000    5735.833    1.42e+04
GarageArea    -2119.0161   2141.111     -0.990      0.323   -6320.093    2082.061
TotalBsmtSF    -252.7962   1851.231     -0.137      0.891   -3885.099    3379.506
1stFlrSF       1.514e+04   7961.194      1.902      0.057    -479.161    3.08e+04
FullBath       -174.7788   1403.452     -0.125      0.901   -2928.495    2578.937
TotRmsAbvGrd    617.2468   1738.003      0.355      0.723   -2792.892    4027.385
YearBuilt      2007.4772   2194.324      0.915      0.360   -2298.009    6312.963
YearRemodAdd   3111.9881   1364.490      2.281      0.023     434.721    5789.255
MasVnrArea     6893.9680   1183.228      5.826      0.000    4572.354    9215.582
GarageYrBlt   -1064.6937   1676.606     -0.635      0.526   -4354.364    2224.977
Fireplaces     4040.5083   1187.122      3.404      0.001    1711.254    6369.762
BsmtFinSF1     5355.6560   1394.059      3.842      0.000    2620.370    8090.942
LotFrontage   -4663.7266   1169.479     -3.988      0.000   -6958.362   -2369.091
WoodDeckSF     2100.9592   1002.205      2.096      0.036     134.531    4067.387
2ndFlrSF       1.332e+04   8884.050      1.500      0.134   -4108.536    3.08e+04
OpenPorchSF    1017.4354    999.478      1.018      0.309    -943.641    2978.512
MSZoning      -1603.4096   1642.609     -0.976      0.329   -4826.376    1619.557
LotShape      -1695.4655    693.916     -2.443      0.015   -3056.999    -333.932
LandContour    1323.5809   1319.793      1.003      0.316   -1265.987    3913.148
LotConfig       340.2432    591.455      0.575      0.565    -820.252    1500.738
Neighborhood    107.6946    165.343      0.651      0.515    -216.725     432.115
Condition1      301.7947   1136.684      0.266      0.791   -1928.495    2532.085
Condition2      836.1264   3246.436      0.258      0.797   -5533.710    7205.963
BldgType      -7227.0688    909.439     -7.947      0.000   -9011.480   -5442.658
HouseStyle    -1982.5800    617.936     -3.208      0.001   -3195.033    -770.126
RoofStyle      2238.2584   1185.607      1.888      0.059     -88.023    4564.540
RoofMatl       5962.1670   1557.433      3.828      0.000    2906.326    9018.008
Exterior1st    -891.3271    587.184     -1.518      0.129   -2043.441     260.787
Exterior2nd     183.4695    537.652      0.341      0.733    -871.459    1238.398
MasVnrType     5529.1918   1685.252      3.281      0.001    2222.556    8835.827
ExterQual     -8241.6988   2112.965     -3.901      0.000   -1.24e+04   -4095.847
ExterCond      -153.9512   1326.171     -0.116   

In [25]:
col_to_drop = ols.pvalues.sort_values(ascending=False).index[0]

In [26]:
X = X.drop(labels=[col_to_drop],axis=1)
Y = trd[["SalePrice"]]
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.2,random_state=31)
from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst).fit()
ols.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.854
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     118.7
Date:                Fri, 04 Mar 2022   Prob (F-statistic):               0.00
Time:                        22:56:07   Log-Likelihood:                -13680.
No. Observations:                1168   AIC:                         2.747e+04
Df Residuals:                    1112   BIC:                         2.775e+04
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          3.222e+05   2.81e+04     11.446      0.000    2.67e+05    3.77e+05
OverallQual    1.634e+04   1681.730      9.718      0.000     1.3e+04    1.96e+04
GrLivArea      7395.5079   1.07e+04      0.694      0.488   -1.35e+04    2.83e+04
GarageCars     9960.6891   2156.046      4.620      0.000    5730.312    1.42e+04
GarageArea    -2112.2698   2139.373     -0.987      0.324   -6309.932    2085.392
TotalBsmtSF    -266.8425   1846.453     -0.145      0.885   -3889.766    3356.081
1stFlrSF       1.512e+04   7955.139      1.900      0.058    -490.536    3.07e+04
FullBath       -167.1599   1401.295     -0.119      0.905   -2916.640    2582.321
TotRmsAbvGrd    616.1720   1737.207      0.355      0.723   -2792.402    4024.746
YearBuilt      1963.8765   2160.982      0.909      0.364   -2276.186    6203.939
YearRemodAdd   3122.6480   1360.792      2.295      0.022     452.638    5792.658
MasVnrArea     6895.6518   1182.614      5.831      0.000    4575.245    9216.059
GarageYrBlt   -1062.1196   1675.715     -0.634      0.526   -4350.040    2225.801
Fireplaces     4038.1358   1186.419      3.404      0.001    1710.263    6366.009
BsmtFinSF1     5360.5417   1392.806      3.849      0.000    2627.718    8093.365
LotFrontage   -4667.9036   1168.406     -3.995      0.000   -6960.433   -2375.374
WoodDeckSF     2104.4190   1001.317      2.102      0.036     139.735    4069.103
2ndFlrSF       1.328e+04   8872.971      1.497      0.135   -4128.122    3.07e+04
OpenPorchSF    1025.8589    996.398      1.030      0.303    -929.173    2980.891
MSZoning      -1593.6422   1639.725     -0.972      0.331   -4810.946    1623.662
LotShape      -1698.3077    693.176     -2.450      0.014   -3058.388    -338.227
LandContour    1324.7642   1319.168      1.004      0.315   -1263.575    3913.103
LotConfig       339.4426    591.153      0.574      0.566    -820.458    1499.343
Neighborhood    107.5887    165.267      0.651      0.515    -216.682     431.860
Condition1      295.1064   1134.719      0.260      0.795   -1931.326    2521.539
Condition2      849.2042   3243.041      0.262      0.793   -5513.965    7212.373
BldgType      -7232.1499    907.982     -7.965      0.000   -9013.700   -5450.599
HouseStyle    -1983.6000    617.600     -3.212      0.001   -3195.392    -771.808
RoofStyle      2233.8633   1184.477      1.886      0.060     -90.198    4557.925
RoofMatl       5965.6838   1556.447      3.833      0.000    2911.780    9019.588
Exterior1st    -892.8610    586.775     -1.522      0.128   -2044.171     258.449
Exterior2nd     183.6965    537.410      0.342      0.733    -870.756    1238.149
MasVnrType     5521.4078   1683.170      3.280      0.001    2218.860    8823.956
ExterQual     -8261.6313   2105.043     -3.925      0.000   -1.24e+04   -4131.328
Foundation     -221.6638   1757.635     -0.126   

In [27]:
cols_to_drop = ols.pvalues.sort_values(ascending=False).index[0]

In [28]:
cols_to_drop

'FullBath'

In [29]:
col_to_drop = ols.pvalues.sort_values(ascending=False).index[0]
print(cols_to_drop)
X = X.drop(labels=[cols_to_drop],axis=1)
Y = trd[["SalePrice"]]
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.2,random_state=31)
from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst).fit()
print(ols.summary())

FullBath
                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.854
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     121.0
Date:                Fri, 04 Mar 2022   Prob (F-statistic):               0.00
Time:                        22:56:55   Log-Likelihood:                -13680.
No. Observations:                1168   AIC:                         2.747e+04
Df Residuals:                    1113   BIC:                         2.775e+04
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          3.222e+05   2.81e+04  

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [30]:
X.columns

Index(['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF',
       '1stFlrSF', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'GarageYrBlt', 'Fireplaces', 'BsmtFinSF1', 'LotFrontage', 'WoodDeckSF',
       '2ndFlrSF', 'OpenPorchSF', 'MSZoning', 'LotShape', 'LandContour',
       'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
       'MasVnrType', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive',
       'PoolQC', 'Fence', 'SaleType', 'SaleCondition'],
      dtype='object')

In [31]:
X.corr()

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,TotRmsAbvGrd,YearBuilt,YearRemodAdd,MasVnrArea,...,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,SaleType,SaleCondition
OverallQual,1.000000,0.593007,0.600671,0.562022,0.537808,0.476224,0.427452,0.572323,0.550684,0.410238,...,0.064349,-0.297995,-0.538529,0.090562,0.144988,0.229393,-0.074345,0.017985,-0.060472,0.193703
GrLivArea,0.593007,1.000000,0.467247,0.468997,0.454868,0.566024,0.825489,0.199010,0.287389,0.389893,...,0.169837,-0.158783,-0.345646,0.014458,0.060616,0.080195,-0.160523,-0.021475,-0.041480,0.080729
GarageCars,0.600671,0.467247,1.000000,0.882475,0.434585,0.439317,0.362289,0.537850,0.420622,0.363778,...,0.111637,-0.116844,-0.475862,0.093316,0.077340,0.284598,-0.037682,0.028115,-0.068295,0.184943
GarageArea,0.562022,0.468997,0.882475,1.000000,0.486665,0.489782,0.337822,0.478954,0.371600,0.372567,...,0.064570,-0.102921,-0.416669,0.101302,0.086499,0.284885,-0.055290,0.024052,-0.090670,0.181284
TotalBsmtSF,0.537808,0.454868,0.434585,0.486665,1.000000,0.819530,0.285573,0.391452,0.291066,0.362452,...,0.000242,-0.329053,-0.363421,0.087440,0.106404,0.210346,-0.068009,0.053234,-0.094983,0.110853
1stFlrSF,0.476224,0.566024,0.439317,0.489782,0.819530,1.000000,0.409516,0.281986,0.240379,0.342160,...,0.045741,-0.320468,-0.323798,0.084884,0.100718,0.163412,-0.070055,0.014630,-0.118436,0.075893
TotRmsAbvGrd,0.427452,0.825489,0.362289,0.337822,0.285573,0.409516,1.000000,0.095589,0.191740,0.280027,...,0.118717,-0.088995,-0.240464,-0.002250,0.017801,-0.003618,-0.070852,-0.036973,-0.041252,0.070660
YearBuilt,0.572323,0.199010,0.537850,0.478954,0.391452,0.281986,0.095589,1.000000,0.592855,0.314745,...,0.100827,-0.443241,-0.590044,0.303839,0.246576,0.427561,0.013066,0.086481,-0.052425,0.201044
YearRemodAdd,0.550684,0.287389,0.420622,0.371600,0.291066,0.240379,0.191740,0.592855,1.000000,0.179187,...,0.012054,-0.271764,-0.456786,0.130241,0.150579,0.168457,-0.015982,0.049834,0.019647,0.221687
MasVnrArea,0.410238,0.389893,0.363778,0.372567,0.362452,0.342160,0.280027,0.314745,0.179187,1.000000,...,0.060404,-0.158463,-0.272714,0.106678,0.068356,0.145145,0.015567,0.016810,-0.055715,0.087729


In [33]:
from sklearn.linear_model import Lasso
ls = Lasso()
tg = {"alpha":[0.99,0.98,0.97,1.01,1.02,1.03]}
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(ls,tg,scoring = "neg_mean_squared_error",cv=5)
cvmodel = cv.fit(xtrain,ytrain)
cvmodel.best_params_

{'alpha': 1.03}

In [34]:
ls = Lasso(alpha=1.03)
model = ls.fit(X,Y)
pred = model.predict(pred_set[list(X.columns)])

In [35]:
pred

array([213247.81211914, 276416.38243722, 289001.85539394, ...,
       243965.9300117 , 237327.90486854, 352142.84236346])

In [36]:
tsd["predicted_sale_price"]=pred

In [37]:
tsd

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,predicted_sale_price
0,1461,20,RH,80.0,11622,Pave,Grvl,Reg,Lvl,AllPub,...,0,Ex,MnPrv,Shed,0,6,2010,WD,Normal,213247.812119
1,1462,20,RL,81.0,14267,Pave,Grvl,IR1,Lvl,AllPub,...,0,Ex,MnPrv,Gar2,12500,6,2010,WD,Normal,276416.382437
2,1463,60,RL,74.0,13830,Pave,Grvl,IR1,Lvl,AllPub,...,0,Ex,MnPrv,Shed,0,3,2010,WD,Normal,289001.855394
3,1464,60,RL,78.0,9978,Pave,Grvl,IR1,Lvl,AllPub,...,0,Ex,MnPrv,Shed,0,6,2010,WD,Normal,302129.092194
4,1465,120,RL,43.0,5005,Pave,Grvl,IR1,HLS,AllPub,...,0,Ex,MnPrv,Shed,0,1,2010,WD,Normal,297554.891972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,Grvl,Reg,Lvl,AllPub,...,0,Ex,MnPrv,Shed,0,6,2006,WD,Normal,177071.137036
1455,2916,160,RM,21.0,1894,Pave,Grvl,Reg,Lvl,AllPub,...,0,Ex,MnPrv,Shed,0,4,2006,WD,Abnorml,170923.357655
1456,2917,20,RL,160.0,20000,Pave,Grvl,Reg,Lvl,AllPub,...,0,Ex,MnPrv,Shed,0,9,2006,WD,Abnorml,243965.930012
1457,2918,85,RL,62.0,10441,Pave,Grvl,Reg,Lvl,AllPub,...,0,Ex,MnPrv,Shed,700,7,2006,WD,Normal,237327.904869
